# Hierarchically Deep Convolutional Neural Network For Architecture Image Recognition

## Setup and Imports

**Import Packages**

In [1]:
import tensorflow.keras as kr
import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

from sklearn.model_selection import train_test_split

from random import randint
import time
import os

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
if not os.path.exists('models/tf'):
    os.mkdir('models/tf')

**Define Global Variables**

In [4]:
# The number of coarse categories
coarse_categories = 9

# The number of fine categories
fine_categories = 25

## Import and Preprocess Dataset

In [5]:
import random
import keras_preprocessing.image

def load_and_crop_img(path, grayscale=False, color_mode='rgb', target_size=None,
             interpolation='nearest'):
    """
    Wraps keras_preprocessing.image.utils.loag_img() and adds cropping.
    Cropping method enumarated in interpolation
    # Arguments
        path: Path to image file.
        color_mode: One of "grayscale", "rgb", "rgba". Default: "rgb".
            The desired image format.
        target_size: Either `None` (default to original size)
            or tuple of ints `(img_height, img_width)`.
        interpolation: Interpolation and crop methods used to resample and crop the image
            if the target size is different from that of the loaded image.
            Methods are delimited by ":" where first part is interpolation and second is crop
            e.g. "lanczos:random".
            Supported interpolation methods are "nearest", "bilinear", "bicubic", "lanczos",
            "box", "hamming" By default, "nearest" is used.
            Supported crop methods are "none", "center", "random".
    # Returns
        A PIL Image instance.
    # Raises
        ImportError: if PIL is not available.
        ValueError: if interpolation method is not supported.
    """

    # Decode interpolation string. Allowed Crop methods: none, center, random
    interpolation, crop = interpolation.split(":") if ":" in interpolation else (interpolation, "none")  

    if crop == "none":
        return keras_preprocessing.image.utils.load_img(path, 
                                            grayscale=grayscale, 
                                            color_mode=color_mode, 
                                            target_size=target_size,
                                            interpolation=interpolation)

    # Load original size image using Keras
    img = keras_preprocessing.image.utils.load_img(path, 
                                            grayscale=grayscale, 
                                            color_mode=color_mode, 
                                            target_size=None, 
                                            interpolation=interpolation)

    # Crop fraction of total image
    crop_fraction = 0.875
    target_width = target_size[1]
    target_height = target_size[0]

    if target_size is not None:        
        if img.size != (target_width, target_height):

            if crop not in ["center", "random"]:
                raise ValueError('Invalid crop method {} specified.', crop)

            if interpolation not in keras_preprocessing.image.utils._PIL_INTERPOLATION_METHODS:
                raise ValueError(
                    'Invalid interpolation method {} specified. Supported '
                    'methods are {}'.format(interpolation,
                        ", ".join(keras_preprocessing.image.utils._PIL_INTERPOLATION_METHODS.keys())))
            
            resample = keras_preprocessing.image.utils._PIL_INTERPOLATION_METHODS[interpolation]

            width, height = img.size

            # Resize keeping aspect ratio
            # result shold be no smaller than the targer size, include crop fraction overhead
            target_size_before_crop = (target_width/crop_fraction, target_height/crop_fraction)
            ratio = max(target_size_before_crop[0] / width, target_size_before_crop[1] / height)
            target_size_before_crop_keep_ratio = int(width * ratio), int(height * ratio)
            img = img.resize(target_size_before_crop_keep_ratio, resample=resample)

            width, height = img.size

            if crop == "center":
                left_corner = int(round(width/2)) - int(round(target_width/2))
                top_corner = int(round(height/2)) - int(round(target_height/2))
                return img.crop((left_corner, top_corner, left_corner + target_width, top_corner + target_height))
            elif crop == "random":
                left_shift = random.randint(0, int((width - target_width)))
                down_shift = random.randint(0, int((height - target_height)))
                return img.crop((left_shift, down_shift, target_width + left_shift, target_height + down_shift))

    return img
  
keras_preprocessing.image.iterator.load_img = load_and_crop_img

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input

fine_datagen = tf.keras.preprocessing.image.ImageDataGenerator(    
    rotation_range=20,
    horizontal_flip=True,
    samplewise_std_normalization = True)
fine_dir = "dataset"
fine_img = fine_datagen.flow_from_directory(fine_dir,target_size=(64,64), batch_size=128, interpolation = 'lanczos:center')

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 10113 images belonging to 2 classes.


In [7]:
from tqdm import tqdm
fine_img.reset()
X_fine, y_fine = next(fine_img)
for i in tqdm(range(len(fine_img))-1): # 1st batch is already fetched before the for loop.
  img, label = next(fine_img)
  X_fine = np.append(X_fine, img, axis=0)
  y_fine = np.append(y_fine, label, axis=0)
print(X_fine.shape, y_fine.shape)

100%|██████████| 79/79 [12:27<00:00,  9.46s/it]


**Split Training set into Training and Validation sets**

In [17]:
x_train, x_val, y_train, y_val = train_test_split(X_fine, y_fine, test_size=.2,shuffle=True)

In [11]:
# import matplotlib.pyplot as plt
# from IPython.core.display import display, HTML

# for i in range(0,10):
#     image = x_train[i]
#     plt.imshow(image)
#     plt.show()
#     print(np.where(y_train[i] == 1)[0])


# for i in range(0,10):
#     image = x_val[i]
#     plt.imshow(image)
#     plt.show()
#     print(np.where(y_train[i] == 1)[0])

In [12]:
# y_fine_eight = y_fine[np.argwhere(y_fine==1)[:,1] < 8]
# X_fine_eight = X_fine[np.argwhere(y_fine==1)[:,1] < 8]
# x_train_eight, x_val_eight, y_train_eight, y_val_eight = train_test_split(X_fine_eight, y_fine_eight, test_size=.1,shuffle=True)

**Constructing CNN**

In [13]:
from keras import optimizers
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Flatten, Dense, BatchNormalization
from keras.models import Model

in_layer = Input(shape=(64, 64, 3), dtype='float32', name='main_input')

net = Conv2D(64, 3, strides=1, padding='same', activation='relu')(in_layer)
net = BatchNormalization()(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(128, 3, strides=1, padding='same', activation='relu')(net)
net = BatchNormalization()(net)
# net = Dropout(.3)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(256, 3, strides=1, padding='same', activation='relu')(net)
net = BatchNormalization()(net)
# net = Dropout(.4)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(512, 3, strides=1, padding='same', activation='relu')(net)
net = BatchNormalization()(net)
# net = Dropout(.5)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Flatten()(net)

net = Dense(1024, activation='relu')(net)
net = BatchNormalization()(net)
net = Dropout(.3)(net)
net = Dense(25, activation='softmax')(net)

Instructions for updating:
Colocations handled automatically by placer.


**Compile Model**

In [14]:
model = Model(inputs=in_layer,outputs=net)
adam_coarse = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer= adam_coarse, loss='categorical_crossentropy', metrics=['accuracy'])
tbCallBack = kr.callbacks.TensorBoard(log_dir='./data/graph/relu_drop/', histogram_freq=0, write_graph=True, write_images=True)

**Train Model**

In [15]:
batch = 32

In [18]:
index= 0
step = 5
stop = 20


while index < stop:
    model.fit(x_train, y_train, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val), callbacks=[tbCallBack])
    index += step
    model.save_weights('models/tf/model_coarse'+str(index))

save_index = index

ValueError: A target array with shape (8090, 2) was passed for an output of shape (None, 25) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.